## Data Preprocessing
This notebook handles the preprocessing of the dataset that is used to train the model.
<br><br>

### __Dataset Details__
The data consists of labeled images of rocks and minerals organized by their respective classes: Igneous, Sedimentary, and Metamorphic. <br><br>
There are two sets of data, the [__training data__](#training-data-processing) and the [__testing data__](#test-data-processing). Each set follows the same structure outlined above, with the only difference coming from how the images are preprocessed. <br><br>

### __Preprocessing Transformations__
#### Resizing
...
<br><br>

#### Augmentation
Both the `.RandomHorizontalFlip()` and `.RandomRotation()` functions are used to augment the images, providing the model with a greater variety of training data from a limited orignal dataset. 
<br><br>

#### Tensor Conversion
...
<br><br>

#### Normalization
...

In [ ]:
# Using PyTorch to preprocess images
from torchvision import transforms
from torchvision import datasets

### Training Data Processing
The preprocessing consists of:
- Image Resizing
- Image Augmentation
- PyTorch Tensor Conversion
- Normalization

In [ ]:
training_data_processing = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

### Test Data Processing
Test data is processed similary to the training data, with the exception of augmentation.

In [ ]:
test_data_processing = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])